ML MODELLING

In [13]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [14]:
# Sample mock data for training
data = {
    'age': [25, 45, 62, 34, 50, 28, 40],
    'smoker': [0, 1, 1, 0, 1, 0, 0],
    'medical_report_uploaded': [1, 0, 0, 1, 1, 1, 0],
    'existing_conditions': [0, 1, 2, 1, 3, 0, 2],
    'occupation_risk': [0, 1, 1, 0, 1, 0, 0],
    'decision': [1, 2, 0, 1, 2, 1, 0]  # 0: Reject, 1: Accept, 2: Needs Info
}

df = pd.DataFrame(data)
X = df[['age', 'smoker', 'medical_report_uploaded', 'existing_conditions', 'occupation_risk']]
y = df['decision']


In [15]:
model = DecisionTreeClassifier()
model.fit(X, y)


DecisionTreeClassifier()

In [16]:
def underwriting_decision(app):
    # Extract features for model
    features = np.array([
        app['age'],
        int(app['smoker']),
        int(app['medical_report_uploaded']),
        len(app['existing_conditions']),
        1 if app['occupation'] in {"Firefighter", "Pilot", "Construction Worker"} else 0
    ]).reshape(1, -1)

    # Predict using trained model
    pred = model.predict(features)[0]

    # Map prediction to decision
    mapping = {0: "Reject", 1: "Accept", 2: "Needs Additional Information"}
    return mapping[pred]


In [17]:
app = {
    "name": "Jane Doe",
    "age": 42,
    "occupation": "Accountant",
    "sum_assured": 3000000,
    "smoker": False,
    "alcohol": "low",
    "existing_conditions": ["diabetes"],
    "medical_report_uploaded": True
}

decision = underwriting_decision(app)
print(f"Decision for {app['name']}: {decision}")


Decision for Jane Doe: Reject


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Risk Assessment for New Policy Applications

In [12]:
application_data = {
  "application_id": "A123",
  "name": "John Doe",
  "age": 45,
  "income": 75000,
  "smoker": False,
  "medical_conditions": ["hypertension"],
  "requested_coverage": 500000,
  "existing_policies": 1
}

In [2]:
# Define risk scoring rules and weights
risk_rules = {
    "age": [
        {"range": (0, 30), "score": 5},
        {"range": (31, 50), "score": 10},
        {"range": (51, 70), "score": 15},
        {"range": (71, 100), "score": 20},
    ],
    "income": [
        {"range": (0, 30000), "score": 20},
        {"range": (30001, 70000), "score": 10},
        {"range": (70001, float('inf')), "score": 5},
    ],
    "smoker": {True: 25, False: 0},
    "medical_conditions": {
        "hypertension": 15,
        "diabetes": 20,
        "heart disease": 30,
        "cancer": 50,
        "none": 0
    },
    "requested_coverage": [
        {"range": (0, 100000), "score": 5},
        {"range": (100001, 500000), "score": 10},
        {"range": (500001, float('inf')), "score": 15},
    ],
    "existing_policies": [
        {"range": (0, 0), "score": 10},
        {"range": (1, 2), "score": 5},
        {"range": (3, float('inf')), "score": 0},
    ]
}

# Document the rules (for demonstration purposes, stored in a dictionary)
risk_rules_documentation = """
Risk Scoring Rules:

Age:
- 0-30 years: 5 points
- 31-50 years: 10 points
- 51-70 years: 15 points
- 71+ years: 20 points

Income:
- $0-30,000: 20 points
- $30,001-70,000: 10 points
- $70,001+: 5 points

Smoker:
- Yes: 25 points
- No: 0 points

Medical Conditions:
- Hypertension: 15 points
- Diabetes: 20 points
- Heart Disease: 30 points
- Cancer: 50 points
- None: 0 points (if no listed conditions, assume none for initial scoring)

Requested Coverage:
- $0-100,000: 5 points
- $100,001-500,000: 10 points
- $500,001+: 15 points

Existing Policies:
- 0 policies: 10 points
- 1-2 policies: 5 points
- 3+ policies: 0 points
"""

print(risk_rules_documentation)


Risk Scoring Rules:

Age:
- 0-30 years: 5 points
- 31-50 years: 10 points
- 51-70 years: 15 points
- 71+ years: 20 points

Income:
- $0-30,000: 20 points
- $30,001-70,000: 10 points
- $70,001+: 5 points

Smoker:
- Yes: 25 points
- No: 0 points

Medical Conditions:
- Hypertension: 15 points
- Diabetes: 20 points
- Heart Disease: 30 points
- Cancer: 50 points
- None: 0 points (if no listed conditions, assume none for initial scoring)

Requested Coverage:
- $0-100,000: 5 points
- $100,001-500,000: 10 points
- $500,001+: 15 points

Existing Policies:
- 0 policies: 10 points
- 1-2 policies: 5 points
- 3+ policies: 0 points



In [3]:
def calculate_risk_score(application_data, risk_rules):
    """Calculates the risk score for an insurance application based on defined rules."""
    total_risk_score = 0

    # Score age
    age = application_data.get("age")
    if age is not None:
        for rule in risk_rules["age"]:
            if rule["range"][0] <= age <= rule["range"][1]:
                total_risk_score += rule["score"]
                break

    # Score income
    income = application_data.get("income")
    if income is not None:
        for rule in risk_rules["income"]:
            if rule["range"][0] <= income <= rule["range"][1]:
                total_risk_score += rule["score"]
                break

    # Score smoker status
    smoker = application_data.get("smoker")
    if smoker is not None:
        total_risk_score += risk_rules["smoker"].get(smoker, 0)

    # Score medical conditions
    medical_conditions = application_data.get("medical_conditions", [])
    for condition in medical_conditions:
        total_risk_score += risk_rules["medical_conditions"].get(condition, 0)

    # Score requested coverage
    requested_coverage = application_data.get("requested_coverage")
    if requested_coverage is not None:
        for rule in risk_rules["requested_coverage"]:
            if rule["range"][0] <= requested_coverage <= rule["range"][1]:
                total_risk_score += rule["score"]
                break

    # Score existing policies
    existing_policies = application_data.get("existing_policies")
    if existing_policies is not None:
        for rule in risk_rules["existing_policies"]:
            if rule["range"][0] <= existing_policies <= rule["range"][1]:
                total_risk_score += rule["score"]
                break

    return total_risk_score

# Demonstrate the function with the provided application_data
risk_score = calculate_risk_score(application_data, risk_rules)
print(f"The calculated risk score for the application is: {risk_score}")

The calculated risk score for the application is: 45


In [4]:
underwriting_rules = {
    "risk_score_bands": [
        {"range": (0, 20), "decision": "Accept", "reason": "Low risk score"},
        {"range": (21, 50), "decision": "Require More Info", "reason": "Moderate risk score"},
        {"range": (51, float('inf')), "decision": "Reject", "reason": "High risk score"}
    ],
    "override_rules": [
        {"condition": lambda app_data: "cancer" in app_data.get("medical_conditions", []), "decision": "Reject", "reason": "Applicant has a history of cancer"},
        {"condition": lambda app_data: app_data.get("requested_coverage", 0) > 1000000, "decision": "Require More Info", "reason": "High requested coverage requires additional review"}
    ]
}

# Document the rules (for demonstration purposes, stored in a dictionary)
underwriting_rules_documentation = """
Underwriting Rules:

Risk Score Bands:
- 0-20: Accept (Low risk score)
- 21-50: Require More Info (Moderate risk score)
- 51+: Reject (High risk score)

Override Rules:
- If applicant has 'cancer' as a medical condition: Reject (Applicant has a history of cancer)
- If requested coverage is > $1,000,000: Require More Info (High requested coverage requires additional review)
"""

print(underwriting_rules_documentation)


Underwriting Rules:

Risk Score Bands:
- 0-20: Accept (Low risk score)
- 21-50: Require More Info (Moderate risk score)
- 51+: Reject (High risk score)

Override Rules:
- If applicant has 'cancer' as a medical condition: Reject (Applicant has a history of cancer)
- If requested coverage is > $1,000,000: Require More Info (High requested coverage requires additional review)



In [5]:
def make_underwriting_decision(application_data, risk_score, underwriting_rules):
    """
    Applies underwriting rules to determine the decision for an insurance application.

    Args:
        application_data (dict): The application data.
        risk_score (int): The calculated risk score.
        underwriting_rules (dict): The defined underwriting rules.

    Returns:
        dict: A dictionary containing the 'decision' and 'reason'.
    """
    # Check override rules first
    for rule in underwriting_rules.get("override_rules", []):
        if rule["condition"](application_data):
            return {"decision": rule["decision"], "reason": rule["reason"]}

    # If no override rule is met, check risk score bands
    for band in underwriting_rules.get("risk_score_bands", []):
        if band["range"][0] <= risk_score <= band["range"][1]:
            return {"decision": band["decision"], "reason": band["reason"]}

    # Default decision if risk score is outside defined bands (should not happen)
    return {"decision": "Error", "reason": "Risk score outside defined bands"}

# Demonstrate the function with the provided data
underwriting_decision = make_underwriting_decision(application_data, risk_score, underwriting_rules)
print(f"Underwriting Decision: {underwriting_decision['decision']}")
print(f"Reason: {underwriting_decision['reason']}")

Underwriting Decision: Require More Info
Reason: Moderate risk score


In [6]:
# Call the calculate_risk_score function
risk_score = calculate_risk_score(application_data, risk_rules)

# Call the make_underwriting_decision function
underwriting_decision = make_underwriting_decision(application_data, risk_score, underwriting_rules)

# Print the application data and the final underwriting decision
print("Application Data:")
for key, value in application_data.items():
    print(f"  {key}: {value}")
print("\nUnderwriting Decision:")
print(f"  Decision: {underwriting_decision['decision']}")
print(f"  Reason: {underwriting_decision['reason']}")

Application Data:
  application_id: A123
  name: John Doe
  age: 45
  income: 75000
  smoker: False
  medical_conditions: ['hypertension']
  requested_coverage: 500000
  existing_policies: 1

Underwriting Decision:
  Decision: Require More Info
  Reason: Moderate risk score


In [9]:
a_application_data = {
  "application_id": "A124",
  "name": "Jane Smith",
  "age": 25,
  "income": 80000,
  "smoker": False,
  "medical_conditions": [],
  "requested_coverage": 50000,
  "existing_policies": 2
}

# Process the application
risk_score_accept = calculate_risk_score(a_application_data, risk_rules)
underwriting_decision_accept = make_underwriting_decision(a_application_data, risk_score_accept, underwriting_rules)

# Print the application data and the final underwriting decision
print("Application Data (Accept Example):")
for key, value in a_application_data.items():
    print(f"  {key}: {value}")
print("\nUnderwriting Decision:")
print(f"  Decision: {underwriting_decision_accept['decision']}")
print(f"  Reason: {underwriting_decision_accept['reason']}")

Application Data (Accept Example):
  application_id: A124
  name: Jane Smith
  age: 25
  income: 80000
  smoker: False
  medical_conditions: []
  requested_coverage: 50000
  existing_policies: 2

Underwriting Decision:
  Decision: Accept
  Reason: Low risk score


In [10]:
r_application_data_override = {
  "application_id": "A125",
  "name": "Peter Jones",
  "age": 60,
  "income": 40000,
  "smoker": True,
  "medical_conditions": ["cancer"],
  "requested_coverage": 200000,
  "existing_policies": 0
}

# Process the application
risk_score_reject_override = calculate_risk_score(r_application_data_override, risk_rules)
underwriting_decision_reject_override = make_underwriting_decision(r_application_data_override, risk_score_reject_override, underwriting_rules)

# Print the application data and the final underwriting decision
print("Application Data (Reject Example - Override):")
for key, value in r_application_data_override.items():
    print(f"  {key}: {value}")
print("\nUnderwriting Decision:")
print(f"  Decision: {underwriting_decision_reject_override['decision']}")
print(f"  Reason: {underwriting_decision_reject_override['reason']}")

Application Data (Reject Example - Override):
  application_id: A125
  name: Peter Jones
  age: 60
  income: 40000
  smoker: True
  medical_conditions: ['cancer']
  requested_coverage: 200000
  existing_policies: 0

Underwriting Decision:
  Decision: Reject
  Reason: Applicant has a history of cancer


In [11]:
r_application_data_score = {
  "application_id": "A126",
  "name": "Mary Williams",
  "age": 75,
  "income": 25000,
  "smoker": True,
  "medical_conditions": ["diabetes", "heart disease"],
  "requested_coverage": 600000,
  "existing_policies": 3
}

# Process the application
risk_score_reject_score = calculate_risk_score(r_application_data_score, risk_rules)
underwriting_decision_reject_score = make_underwriting_decision(r_application_data_score, risk_score_reject_score, underwriting_rules)

# Print the application data and the final underwriting decision
print("Application Data (Reject Example - High Score):")
for key, value in r_application_data_score.items():
    print(f"  {key}: {value}")
print("\nUnderwriting Decision:")
print(f"  Decision: {underwriting_decision_reject_score['decision']}")
print(f"  Reason: {underwriting_decision_reject_score['reason']}")

Application Data (Reject Example - High Score):
  application_id: A126
  name: Mary Williams
  age: 75
  income: 25000
  smoker: True
  medical_conditions: ['diabetes', 'heart disease']
  requested_coverage: 600000
  existing_policies: 3

Underwriting Decision:
  Decision: Reject
  Reason: High risk score
